In [1]:
!pip install transformers --quiet
!pip install wandb --quiet
!pip install evaluate --quiet
!pip install datasets --quiet

In [2]:
import torch

torch.cuda.is_available()

True

In [3]:
import pandas as pd

all_data = pd.read_csv("/kaggle/input/bert-embeddings-evaluation-dataset/llm_aggregated_data_v3_filtered.csv/llm_aggregated_data_v3_filtered.csv")

In [4]:
all_data.head()

,Unnamed: 0,COUNTRY,INITIAL_TYPIFICATION_OG,INITIAL_TYPIFICATION,FINAL_TYPIFICATION_AGENT,INITIAL_LEVEL_1,INITIAL_LEVEL_2,INITIAL_LEVEL_3,FINAL_LEVEL_1,FINAL_LEVEL_2,FINAL_LEVEL_3,message_data,prompt_embedding,prompt_embedding_v2
0,0,AR,sn_org_mp.sn_org_pc.sn_org_pq_cmp,NaN,sn_org_pwo.sn_org_acam,Unidentified,Unidentified,Unidentified,Problems with the order,Help with another issue,Unidentified,"""[{\""KUSTOMER_CONVERSATION_ID\"": \""61fd4520ab5...",A continuación se presenta un chat entre un cl...,A continuación se presenta un chat entre un cl...
1,1,AR,sn_org_mp.sn_org_pa.sn_org_tpd,NaN,sn_org_pwo.sn_org_acam,Unidentified,Unidentified,Unidentified,Problems with the order,Help with another issue,Unidentified,"""[{\""KUSTOMER_CONVERSATION_ID\"": \""622940a8653...",A continuación se presenta un chat entre un cl...,A continuación se presenta un chat entre un cl...
2,2,AR,sn_org_mp.sn_org_pa.sn_org_pme,NaN,sn_org_pwo.sn_org_io,Unidentified,Unidentified,Unidentified,Problems with the order,Incomplete order,Unidentified,"""[{\""KUSTOMER_CONVERSATION_ID\"": \""622ccfd89a7...",A continuación se presenta un chat entre un cl...,A continuación se presenta un chat entre un cl...
3,3,CO,sn_org_mp.sn_org_pa.sn_org_pi,NaN,sn_org_pwo.sn_org_ona_3,Unidentified,Unidentified,Unidentified,Problems with the order,Order never arrived,Unidentified,"""[{\""KUSTOMER_CONVERSATION_ID\"": \""622d602fad3...",A continuación se presenta un chat entre un cl...,A continuación se presenta un chat entre un cl...
4,4,PE,sn_org_mp.sn_org_pa.sn_org_tpd,NaN,sn_org_pwp.sn_org_giap,Unidentified,Unidentified,Unidentified,Problems with promotions,General information about promotions,Unidentified,"""[{\""KUSTOMER_CONVERSATION_ID\"": \""6230305d00e...",A continuación se presenta un chat entre un cl...,A continuación se presenta un chat entre un cl...


In [5]:
from tqdm import tqdm
tqdm.pandas()

vc = all_data.FINAL_LEVEL_1.value_counts()
min_n_samples = 8000
grouped_classes = vc[vc < min_n_samples].keys().tolist()

def grouped_labels(entry):
    label = entry.FINAL_LEVEL_1
    if label in grouped_classes:
        label = 'other'
    entry['grouped_label'] = label
    return entry

all_data = all_data.progress_apply(grouped_labels, axis=1)
all_data.grouped_label.value_counts()

100%|██████████| 119308/119308 [01:18<00:00, 1517.48it/s]


Problems with the order     22266
Account User                21926
Problems with the RT        20987
Disagree with charge        19143
other                       17503
Problems with promotions     8878
RappiPrime                   8605
Name: grouped_label, dtype: int64

In [6]:
from sklearn.model_selection import train_test_split
import numpy as np

all_data.grouped_label = all_data.grouped_label.astype('category')
train, test = train_test_split(all_data, test_size=0.2, random_state=0)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [7]:

all_data.grouped_label = all_data.grouped_label.astype('category')
labels = all_data.grouped_label.cat.categories.tolist()
id2label = {i:v for i, v in enumerate(labels)}
label2id = {v:i for i, v in enumerate(labels)}

In [8]:
all_data['label'] = all_data.grouped_label.cat.codes.values
all_data.head(3)

,Unnamed: 0,COUNTRY,INITIAL_TYPIFICATION_OG,INITIAL_TYPIFICATION,FINAL_TYPIFICATION_AGENT,INITIAL_LEVEL_1,INITIAL_LEVEL_2,INITIAL_LEVEL_3,FINAL_LEVEL_1,FINAL_LEVEL_2,FINAL_LEVEL_3,message_data,prompt_embedding,prompt_embedding_v2,grouped_label,label
0,0,AR,sn_org_mp.sn_org_pc.sn_org_pq_cmp,NaN,sn_org_pwo.sn_org_acam,Unidentified,Unidentified,Unidentified,Problems with the order,Help with another issue,Unidentified,"""[{\""KUSTOMER_CONVERSATION_ID\"": \""61fd4520ab5...",A continuación se presenta un chat entre un cl...,A continuación se presenta un chat entre un cl...,Problems with the order,4
1,1,AR,sn_org_mp.sn_org_pa.sn_org_tpd,NaN,sn_org_pwo.sn_org_acam,Unidentified,Unidentified,Unidentified,Problems with the order,Help with another issue,Unidentified,"""[{\""KUSTOMER_CONVERSATION_ID\"": \""622940a8653...",A continuación se presenta un chat entre un cl...,A continuación se presenta un chat entre un cl...,Problems with the order,4
2,2,AR,sn_org_mp.sn_org_pa.sn_org_pme,NaN,sn_org_pwo.sn_org_io,Unidentified,Unidentified,Unidentified,Problems with the order,Incomplete order,Unidentified,"""[{\""KUSTOMER_CONVERSATION_ID\"": \""622ccfd89a7...",A continuación se presenta un chat entre un cl...,A continuación se presenta un chat entre un cl...,Problems with the order,4


In [9]:
from datasets import Dataset
data = Dataset.from_dict(all_data[["label", "prompt_embedding"]].to_dict(orient="list"))

In [10]:
data = data.train_test_split(test_size=0.2, seed=0)

In [11]:
data

DatasetDict({
    train: Dataset({
        features: ['label', 'prompt_embedding'],
        num_rows: 95446
    })
    test: Dataset({
        features: ['label', 'prompt_embedding'],
        num_rows: 23862
    })
})

In [12]:
import transformers

In [13]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [14]:
def preprocess_function(examples):
    return tokenizer(examples["prompt_embedding"], truncation=True, padding=True)

tokenized_data = data.map(preprocess_function, batched=True)

  0%|          | 0/96 [00:00<?, ?ba/s]

  0%|          | 0/24 [00:00<?, ?ba/s]

In [15]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True, max_length=512)

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [16]:
import numpy as np
import evaluate

accuracy = evaluate.load('accuracy')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [17]:
import wandb
import os
from kaggle_secrets import UserSecretsClient
os.environ["WANDB_NOTEBOOK_NAME"] = "transformers_for_classification.ipynb"

user_secrets = UserSecretsClient()

# I have saved my API token with "wandb_api" as Label. 
# If you use some other Label make sure to change the same below. 
wandb_api = user_secrets.get_secret("wandb_api") 

wandb.login(key=wandb_api)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [18]:
tokenized_data

DatasetDict({
    train: Dataset({
        features: ['label', 'prompt_embedding', 'input_ids', 'attention_mask'],
        num_rows: 95446
    })
    test: Dataset({
        features: ['label', 'prompt_embedding', 'input_ids', 'attention_mask'],
        num_rows: 23862
    })
})

In [19]:
#tokenized_data = tokenized_data.rename_column("grouped_label", "label")
# tokenized_data = tokenized_data.rename_column("prompt_embedding", "text")
tokenized_data = tokenized_data.remove_columns(['prompt_embedding'])
# tokenized_data = tokenized_data.rename_column("labels", "label")
tokenized_data

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 95446
    })
    test: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 23862
    })
})

In [20]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=len(labels), id2label=id2label, label2id=label2id
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.weight', 'pre_classifier.

In [21]:
training_args = TrainingArguments(
    output_dir="text_classification_v1",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="steps",
    eval_steps=150,
    save_strategy="steps",
    load_best_model_at_end=True,
    report_to="wandb",
    run_name="text_classification_v1"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱  1 training_args = TrainingArguments(                                                          │
│    2 │   output_dir="text_classification_v1",                                                    │
│    3 │   learning_rate=2e-5,                                                                     │
│    4 │   per_device_train_batch_size=32,                                                         │
│ in __init__:111                                                                                  │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1258 in __post_init__      │
│                                                                                                  │
│   1255 │   │   │   │   │   │   │   "--load_best_model_at_end requires the saving steps to be a   │
│   1256 │   │   │   │   │   │   │   f"steps, but found {self.save_steps}, which is not a multipl  │
│   1257 │   │   │   │   │   │   )                                                                 │
│ ❱ 1258 │   │   │   │   raise ValueError(                                                         │
│   1259 │   │   │   │   │   "--load_best_model_at_end requires the saving steps to be a round mu  │
│   1260 │   │   │   │   │   f"steps, but found {self.save_steps}, which is not a round multiple   │
│   1261 │   │   │   │   )                                                                         │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: --load_best_model_at_end requires the saving steps to be a round multiple of the evaluation steps, but 
found 500, which is not a round multiple of 150.

In [ ]:
transformers.logging.set_verbosity_info()
trainer.train()